In [1]:
%pylab inline

import os
import sys
import socket

sys.path.append("/root")
sys.path.append("/home/peter")
sys.path.append("/home/peter/Desktop")
sys.path.append("/home/peter/Desktop/projects")
sys.path.append("/home/peter/Desktop/work")
sys.path.append("/home/peter/code/projects")
sys.path.append("/home/peter/code/work")
sys.path.append("/home/peter/code/common")

from ipywidgets import interact, interactive, fixed, IntSlider, FloatSlider
from IPython.display import clear_output, display, HTML

import cv2
import os
import pickle as pkl
from sklearn import linear_model
from sklearn.decomposition import PCA
import random as rnd
import numpy as np
import matplotlib.pyplot as plt

import theano.tensor as T
import theano as th

from aidevutil import *

import cymysql as sql
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import datetime
import time

Populating the interactive namespace from numpy and matplotlib


Can not use cuDNN on context None: cannot compile with cuDNN. We got this error:
b'/tmp/try_flags_x46bnqkq.c:4:10: fatal error: cudnn.h: No such file or directory\n #include <cudnn.h>\n          ^~~~~~~~~\ncompilation terminated.\n'
Mapped name None to device cuda: GeForce GTX 1050 with Max-Q Design (0000:02:00.0)
Using TensorFlow backend.


vizdoom could not be imported
Pixiedust database opened successfully


In [2]:
from sklearn.preprocessing import scale, robust_scale, normalize

In [3]:
input_source = np.load(open('data_eth_1m_2019.npy','rb'))
to_predict = np.load(open('data_eth_1m_2019_targets.npy','rb'))

to_predict = to_predict[3,:].reshape(-1)

In [4]:
input_source.shape

(176, 225939)

In [5]:
input_source = input_source.T

In [6]:
is_orig = np.copy(input_source)
cp = int(0.8*len(input_source))
test_input_source = input_source[cp:, :]
test_to_predict = to_predict[cp:]
input_source = input_source[0:cp, :]
to_predict = to_predict[0:cp]

In [7]:
test_input_source = test_input_source.T
input_source = input_source.T

In [8]:
input_source.shape, test_input_source.shape

((176, 180751), (176, 45188))

In [9]:
to_predict[0:50]

array([129.79, 129.79, 129.79, 129.79, 129.79, 129.79, 130.44, 130.44,
       130.44, 130.44, 130.44, 130.44, 130.44, 130.35, 130.44, 130.44,
       130.5 , 130.5 , 130.5 , 130.49, 130.65, 130.65, 130.65, 130.65,
       130.65, 131.  , 131.  , 131.  , 130.64, 130.64, 130.64, 130.64,
       130.93, 130.93, 130.93, 130.93, 130.93, 130.93, 130.93, 130.91,
       130.71, 130.73, 130.73, 130.73, 130.73, 130.73, 130.73, 130.73,
       130.73, 130.73])

In [10]:
winlen = 1

In [11]:
sliding_window_jump = 1
predict_time_ahead = 1

In [12]:
def get_partitioned_counts(cl, rr, rt):
    " data, steps_forward, partition_num"
    cl = cl[rr:] - cl[0:-rr]
    cl = hstack([[0]*rr, cl])
    pdown = len(cl[cl < -rt])
    pup = len(cl[cl > rt])
    pmid = len(cl[(cl <= rt) & (cl >= -rt)])
    return pup, pmid, pdown

In [13]:
def partition_data_num(cl, predict_ahead):
    a=[]
    b=[]
    c=[]
    ts = np.linspace(0.001, 10.0, 100000, )
    for x in tqdm(ts):
        au, am, ad = get_partitioned_counts(cl, predict_ahead, x)
        a.append(au)
        b.append(am)
        c.append(ad)
    a, b, c = array(a), array(b), array(c)
    return ts[np.argmin(np.abs(a-b) + np.abs(a-c) + np.abs(b-c))]

In [14]:
# create training set
sep = partition_data_num(to_predict, predict_time_ahead)
sep

0.001

In [15]:
x = []
y = []

ups = []
downs = []
mids = []

for i in tqdm(range(0, input_source.shape[1]-(winlen+100), sliding_window_jump)):
    # form the input
    xs = input_source[:, i:i+winlen]
    #xs = scale(xs, axis=1)
    xs = xs.ravel()

    # for the output
    now = to_predict[i+winlen-1].ravel() # close
    future = to_predict[i+winlen+(predict_time_ahead-1)].ravel() # next close

    ys = future-now
    magn = abs(ys)

    if magn < sep:
        mids.append( (xs, np.array([0, 1, 0])) )
    else:
        if ys < 0:
            downs.append( (xs, np.array([0, 0, 1])) )
        else:
            ups.append( (xs, np.array([1, 0, 0])) )

    if (i%1000)==0:
        clear_output(wait=True)
        print('({}/{})'.format(i, input_source.shape[1]-winlen))

print('Ups: {0} Mids: {1} Downs: {2}'.format(len(ups), len(mids), len(downs)))
print('Separator: %3.5f' % sep)
print()

samplesize = min(len(ups), len(downs), len(mids))
s1 = rnd.sample(ups, samplesize)
s2 = rnd.sample(downs, samplesize)
s3 = rnd.sample(mids, samplesize)
a = s1 + s2 + s3
rnd.shuffle(a)
x = [x[0] for x in a]
y = [x[1] for x in a]

x = np.vstack(x)
y = np.vstack(y)

(180000/180750)

Ups: 28734 Mids: 122777 Downs: 29139
Separator: 0.00100



In [16]:
x.shape, y.shape

((86202, 176), (86202, 3))

In [17]:
x.shape = (x.shape[0], -1)

In [18]:
x.shape

(86202, 176)

In [19]:
#matshow(x[400:500].T); colorbar();

In [20]:
cutpoint = int(0.8 * x.shape[0])
x_train = x[0:cutpoint]
x_test = x[cutpoint:]
y_train = y[0:cutpoint]
y_test = y[cutpoint:]

In [21]:
from sklearn.ensemble import AdaBoostClassifier

In [22]:
# Classifiers
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier 
from sklearn.naive_bayes import GaussianNB
#from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

In [23]:
import sklearn as sk
import time

In [24]:
clf = LogisticRegression()#GradientBoostingClassifier(n_estimators=100, loss='deviance')

In [25]:
%%time

clf.fit(x_train, argmax(y_train, axis=1))

CPU times: user 1min 5s, sys: 49 ms, total: 1min 5s
Wall time: 1min 5s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [26]:
clf.score(x_train, argmax(y_train, axis=1))

0.3810559591653253

In [27]:
clf.score(x_test, argmax(y_test, axis=1))

0.37503625079751757

In [28]:
# Here the model is saved
pkl.dump(clf, open('test.pkl','wb'))

In [ ]:
n_cpu = 1
env = TradingEnv(input_source, to_predict,
                 winlen=winlen, bars_per_episode=180000, traded_amt=traded_amt,
                 commission=commission, slippage=slippage,
                 initial_balance=initial_balance,
                 reward_type='cur_balance',
                 min_ratio_trades = 20,
                 max_position_time = 30,
                 )


observation = env.reset()
done = False
navs = []
acts = []
for i in tqdm(range(env.bars_per_episode)):
    action, nstate = model.predict([observation], state=nstate, deterministic=1)
    acts.append(action)
    observation, reward, done, info = env.step(action)#env.step(action)
    if done:
        break
navs = env.envs[0].returns
kl = []
t = 0
for n in navs:
    t = t + n
    kl.append(t)
plot(kl);